In [60]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
import re
import requests

In [61]:
data = pd.read_csv('../scrapers/pfaf/test_plants.csv')

In [33]:
data

,Family,Genus,Species,CommonName,GrowthRate,HardinessZones,Height,Width,Type,Leaf,...,Soils,pH,Preferences,Tolerances,Habitat,HabitatRange,Edibility,Medicinal,OtherUses,PFAF
1,Apiaceae or Umbelliferae,Bunium,bulbocastanum,"Pig Nut, Earth-nut",NaN,"[4, 5, 6, 7, 8]",0.6,0.3,perennial,NaN,...,"['light', 'medium', 'heavy']","mildly acid, neutral and basic (mildly alkalin...",[' well-drained soil. Suitable pH: mildly acid...,[],Rough grassland and banks on chalk[17].,"Western and Southern Europe, including Britain...",(4 of 5),(1 of 5),(0 of 5),https://pfaf.org/user/Plant.aspx?LatinName=Bun...
2,Lamiaceae or Labiatae,Calamintha,nepeta,Lesser Calamint,NaN,"[5, 6, 7, 8, 9]",0.6,0.8,perennial,NaN,...,"['light', 'medium']",neutral and basic (mildly alkaline) soils and ...,[' well-drained soil. Suitable pH: neutral'],[],"Dry banks, usually on calcareous soils[17].","Europe, including Britain, south from France a...",(3 of 5),(2 of 5),(2 of 5),https://pfaf.org/user/Plant.aspx?LatinName=Cal...
3,Juglandaceae,Juglans,nigra,Black Walnut,fast,"[4, 5, 6, 7, 8, 9]",30,20.0,deciduous,NaN,...,"['light', 'medium', 'heavy']","mildly acid, neutral and basic (mildly alkaline)",[' well-drained soil. Suitable pH: mildly acid...,[],"Rich fertile woods and hillsides[43, 82] in de...","Eastern N. America - Massachusetts to Florida,...",(3 of 5),(3 of 5),(4 of 5),https://pfaf.org/user/Plant.aspx?LatinName=Jug...
4,Juglandaceae,Juglans,regia,"Walnut, English walnut, Persian Walnut,",medium,"[7, 8, 9]",20,20.0,deciduous,NaN,...,"['light', 'medium', 'heavy']","mildly acid, neutral and basic (mildly alkaline)",[' well-drained soil. Suitable pH: mildly acid...,[],"Forests in the Himalayas, preferring a norther...",E. Europe to N. Asia. More or less naturalized...,(4 of 5),(3 of 5),(4 of 5),https://pfaf.org/user/Plant.aspx?LatinName=Jug...
5,Rosaceae,Malus,pumila,"Paradise Apple, Common Apple, Apple Tree",medium,"[3, 4, 5, 6, 7, 8, 9]",7,NaN,deciduous,NaN,...,"['light', 'medium', 'heavy', 'heavy']","mildly acid, neutral and basic (mildly alkaline)",[' well-drained soil'],[],Not known in a truly wild situation[74].,"Europe, including Britain, from Scandanavia so...",(4 of 5),(2 of 5),(3 of 5),https://pfaf.org/user/Plant.aspx?LatinName=Mal...
6,Malvaceae,NaN,Malva alcea,"Vervain mallow, Hollyhock Mallow",fast,"[4, 5, 6, 7, 8]",1.2,0.8,perennial,NaN,...,"['light', 'medium', 'heavy']","mildly acid, neutral and basic (mildly alkaline)",[' well-drained soil. Suitable pH: mildly acid...,[],"Gardens, vineyards and waste places[74].",Europe. An occasional casual in Britain.,(5 of 5),(1 of 5),(2 of 5),https://pfaf.org/user/Plant.aspx?LatinName=+Ma...
7,Alliaceae,Allium,ursinum,Wild Garlic,NaN,"[4, 5, 6, 7, 8]",0.3,0.3,bulb,February to June,...,"['light', 'medium']","mildly acid, neutral and basic (mildly alkaline)",[],[],"Damp soils in woods, copses, valleys and simil...","Much of Europe, including Britain, east to the...",(5 of 5),(3 of 5),(2 of 5),https://pfaf.org/user/Plant.aspx?LatinName=All...
8,Lamiaceae or Labiatae,Agastache,foeniculum,"Anise Hyssop, Blue giant hyssop",NaN,"[4, 5, 6, 7, 8, 9]",0.9,0.4,perennial,NaN,...,"['light', 'medium']","mildly acid, neutral and basic (mildly alkaline)",[' well-drained soil. Suitable pH: mildly acid...,[],"Dry thickets, fields and waste ground[43] on p...","Western N. America - Ontario to Washington, so...",(5 of 5),(1 of 5),(3 of 5),https://pfaf.org/user/Plant.aspx?LatinName=Aga...
9,Fabaceae or Leguminosae,Colutea,arborescens,Bladder Senna,fast,"[4, 5, 6, 7, 8]",3.6,3.0,deciduous,NaN,...,"['light', 'medium', 'heavy']","mildly acid, neutral and basic (mildly alkaline)","[' well-drained soil', 'dry or moist soil']","['drought', 'atmospheric pollution']","Open woods, roadsides, railway banks etc[9], o...",S. Europe - Mediterranean. Naturalized in Brit...,(0 of 5),(2 of 5),(4 of 5),https://pfaf.org/user/Plant.aspx?LatinName=Col...
10,Vitaceae,Vitis,vinifera,"Grape, Wine grape, Purpleleaf Grape, Common Grape",fast,"[6, 7, 8, 9, 1

In [55]:
def get_plant_info(genus, species):

    pfaf_url = f"https://pfaf.org/user/Plant.aspx?LatinName={genus}+{species}"
    headers={"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36"}
    page = requests.get(pfaf_url, headers=headers)
    soup = BeautifulSoup(page.content, 'html.parser')

    # Get the description from which we will extract the characteristics
    # Description in string and list form
    description = soup.find("meta", id="description")['content']
    description_list = description.split()

    ##### Get the characteristics #####

    # deciduous or coniferous / shrub / groundcover etc
    type_idx = description_list.index("is") + 2
    plant_type = " ".join(description_list[type_idx:description_list.index("growing")-1])

    height_idx = description_list.index("growing") + 2
    height = float(description_list[height_idx])

    try:
        growth_rate_idx = description_list.index("rate.") - 1 # growth rate is before "rate."
        growth_rate = description_list[growth_rate_idx]
    except ValueError:
        growth_rate = "" # sometimes no growth rate

    if "by" in description_list[:15]:
        width_idx = description_list.index("by") + 1 # width is after "by"
        width = float(description_list[width_idx])
    else:
        width = ""

    # When the plant leafs out
    leaf = "" # Better to do this before or after if statement?
    if "leaf" in description_list:
        leaf_idx = description_list.index("leaf")
        if description_list[leaf_idx + 1] == "from":
            leaf = " ".join(description_list[leaf_idx + 2: leaf_idx + 5]).strip(",")


    # When the plant is in flower
    flower = ""
    if "flower" in description_list:
        flower_idx = description_list.index("flower")
        if description_list[flower_idx + 1] == "in":
            flower = description_list[flower_idx + 2].strip(",")
        elif description_list[flower_idx + 1] == "from":
            flower = " ".join(description_list[flower_idx + 2: flower_idx + 5]).strip(",")
        else:
            print("Error: Flower not found")

    # When the plant's seed or fruit ripens
    ripen_date = ""
    if "ripen" in description_list:
        tmp_idx = description_list.index("ripen")
        if description_list[tmp_idx + 1] == "in":
            ripen_idx = tmp_idx + 2
            ripen_date = description_list[ripen_idx].strip(".")
        elif description_list[tmp_idx + 1] == "from":
            ripen_idx = tmp_idx + 2
            ripen_date = " ".join(description_list[ripen_idx: ripen_idx + 3]).strip(".")

    # Suitable soil structure (light, medium, heavy)
    soils = ""
    soil_idx = description.find("Suitable for:")
    soil_text = description[soil_idx:description.find(".", soil_idx)]
    soils = re.findall(r"(light|medium|heavy)", soil_text)

    # Suitable pH
    ph = ""
    if "pH:" in description_list:
        ph_idx = description_list.index("pH:") + 1
        ph_list = description_list[ph_idx:]
        # check for soil or soils in the list as end of ph
        if "soils." in ph_list:
            end_idx = ph_list.index("soils.")
        elif "soils" in ph_list:
            end_idx = ph_list.index("soils")
        elif "soil." in ph_list:
            end_idx = ph_list.index("soil.")
        elif "soil" in ph_list:
            end_idx = ph_list.index("soil")
        else:
            ph = "Error: end index not found"
            end_idx=6
    ph = " ".join(ph_list[:end_idx])

    # Monoecious or dioecious ("hermaprodite")
    reproduction = re.findall(r"The species is\s(\w+)", description)[0]

    # What the plant likes
    preferences = re.findall("prefers (.*?)\ and", description)

    # What the plant can tolerate
    tolerances = re.findall("can tolerate (.*?)\.", description)

    # Get information from the table
    table = soup.find("table",{"class":"table table-hover table-striped"})

    common_name = table.find("span", id="ContentPlaceHolder1_lblCommanName").text

    family = table.find("span", id="ContentPlaceHolder1_lblFamily").text

    # String range hardiness (with -) ex: 4-8
    hardiness_range = table.find("span", id="ContentPlaceHolder1_lblUSDAhardiness").text

    # List of hardiness zones
    hardiness_zones = []
    match = re.search(r'(\d+)\-(\d+)', hardiness_range)
    if match:
        zone_min = match.group(1)
        zone_max = match.group(2)

        for zone in range(int(zone_min), int(zone_max) + 1):
            hardiness_zones.append(zone)
    else:
        pass
    
    pollinators = None
    if "pollinated by" in description:
        pollinators_start = description.index("pollinated by") + len("pollinated by") + 1
        pollinators_end = description.index(".", pollinators_start)
        pollinators = description[pollinators_start:pollinators_end].strip().lower()

    # Ecosystems
    habitats = table.find("span", id="ContentPlaceHolder1_txtHabitats").text

    # Native range
    habitat_range = table.find("span", id="ContentPlaceHolder1_lblRange").text

    edibility = table.find("span", id="ContentPlaceHolder1_txtEdrating").text.strip()

    other_uses = table.find("span", id="ContentPlaceHolder1_txtOtherUseRating").text.strip()

    medicinal_rating = table.find("span", id="ContentPlaceHolder1_txtMedRating").text.strip()

    # print(table.prettify())

    # Add Line Space
    # print("")
    print(f'Common name: {common_name} \nFamily: {family} \nHardiness range: {hardiness_range} \
        \nGrowth rate: {growth_rate} \nHeight: {height} meters \nWidth: {width} \nType: {plant_type} \
        \nLeaf: {leaf} \nFlower: {flower} \nRipen date: {ripen_date} \nSoils: {soils} \nSoil text: {soil_text} \npH: {ph}\
        \nReproduction: {reproduction} \nPreferences: {preferences} \nTolerances: {tolerances}\
        \nHabitats: {habitats} \nHabitat range: {habitat_range} \nEdibility: {edibility} \nMedicinal rating: {medicinal_rating} \nOther uses: {other_uses} \
        \nPollinators: {pollinators} \n \nDescription: {description}')

    return family, genus, species, common_name, growth_rate, hardiness_zones, height, width,\
            plant_type, leaf, flower, ripen_date, reproduction, soils, ph, preferences, \
            tolerances, habitats, habitat_range, edibility, medicinal_rating, other_uses, pfaf_url



In [56]:
get_plant_info('Castanea', 'mollissima')

Common name: Chinese Chestnut 
Family: Fagaceae 
Hardiness range: 4-8         
Growth rate: medium 
Height: 25.0 meters 
Width:  
Type: deciduous         
Leaf:  
Flower: July 
Ripen date: October 
Soils: ['light', 'medium', 'heavy'] 
Soil text: Suitable for: light (sandy), medium (loamy) and heavy (clay) soils, prefers  well-drained soil and can grow in nutritionally poor soil 
pH: mildly acid and neutral        
Reproduction: monoecious 
Preferences: [' well-drained soil', 'dry or  moist soil'] 
Tolerances: ['drought']        
Habitats: Woods and forests to 2500 metres[109]. Mountain slopes from near sea level to 2,800 metres[266]. 
Habitat range: E. Asia - China. 
Edibility: (3 of 5) 
Medicinal rating: (2 of 5) 
Other uses: (3 of 5)         
Pollinators: insects 
 
Description: Castanea mollissima is a deciduous Tree growing to 25 m (82ft&nbsp;0in) at a medium rate.  See above for USDA hardiness. It is hardy to UK zone 4.  It is in flower in July, and the seeds ripen in October. The

('Fagaceae',
 'Castanea',
 'mollissima',
 'Chinese Chestnut',
 'medium',
 [4, 5, 6, 7, 8],
 25.0,
 '',
 'deciduous',
 '',
 'July',
 'October',
 'monoecious',
 ['light', 'medium', 'heavy'],
 'mildly acid and neutral',
 [' well-drained soil', 'dry or  moist soil'],
 ['drought'],
 'Woods and forests to 2500 metres[109]. Mountain slopes from near sea level to 2,800 metres[266].',
 'E. Asia - China.',
 '(3 of 5)',
 '(2 of 5)',
 '(3 of 5)',
 'https://pfaf.org/user/Plant.aspx?LatinName=Castanea+mollissima')